In [28]:
import wrds
import pandas_datareader.data as pdd
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [193]:
FETCH_START_DATE = "1999-01-01"
START_DATE = "2000-01-01"
END_DATE = "2024-12-31"

In [252]:
db_num = wrds.Connection()

WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [89]:
constituents_query = """
SELECT 
    gvkey,        -- Compustat's unique company ID
    tic as ticker,
    companyname as company_name,
    indexname,
    fromdate as date_joined
FROM comp.wrds_idx_cst_current
WHERE indexname LIKE '%%S&P 500%%'
AND indexname NOT LIKE '%%Equal Weight%%' 
ORDER BY ticker
"""

print("Fetching current S&P 500 constituents from WRDS...")
df_constituents = db_num.raw_sql(constituents_query)

Fetching current S&P 500 constituents from WRDS...


In [90]:
df_constituents = (
    df_constituents.where(df_constituents.indexname == "S&P 500")
    .dropna()
    .drop(columns=["indexname"])
    .set_index("gvkey")
)
df_constituents.head()

,ticker,company_name,date_joined
gvkey,,,
126554,A,"Agilent Technologies, Inc.",2009-12-18
001690,AAPL,Apple Inc.,2009-12-18
016101,ABBV,AbbVie Inc.,2013-01-02
037460,ABNB,"Airbnb, Inc.",2023-09-18
001078,ABT,Abbott Laboratories,2009-12-18


In [92]:
df_constituents

,ticker,company_name,date_joined
gvkey,,,
126554,A,"Agilent Technologies, Inc.",2009-12-18
001690,AAPL,Apple Inc.,2009-12-18
016101,ABBV,AbbVie Inc.,2013-01-02
037460,ABNB,"Airbnb, Inc.",2023-09-18
001078,ABT,Abbott Laboratories,2009-12-18
...,...,...,...
026367,XYZ,"Block, Inc.",2025-07-23
065417,YUM,"Yum! Brands, Inc.",2009-12-18
144559,ZBH,"Zimmer Biomet Holdings, Inc.",2015-06-29


In [91]:
gvkeys = df_constituents.index.to_list()
gvkeys[:5]

['126554', '001690', '016101', '037460', '001078']

In [ ]:
# We cast to tuple for SQL syntax
gvkeys_tuple = tuple(gvkeys)

In [235]:
price_query = f"""
SELECT
    gvkey,
    tic as ticker, datadate,
    prchd as high_raw, prcld as low_raw, prccd as close_raw,
    cshtrd as volume,
    ajexdi as adj_factor,
    trfd
FROM comp.secd
WHERE gvkey IN {gvkeys_tuple}
AND datadate >= '{FETCH_START_DATE}' AND datadate <= '{END_DATE}'
ORDER BY gvkey, datadate
"""
print("Fetching current S&P 500 constituents price data from WRDS...")
df_prices = db_num.raw_sql(price_query)

Fetching current S&P 500 constituents price data from WRDS...


In [236]:
dates = pd.to_datetime(df_prices["datadate"].unique()).sort_values()
companies = df_prices["gvkey"].unique()
index = pd.MultiIndex.from_product([dates, companies], names=["date", "gvkey"])

In [237]:
df_prices["datadate"] = pd.to_datetime(df_prices["datadate"])
df_prices = (
    df_prices.drop_duplicates(subset=["datadate", "gvkey"], keep="last")
    .set_index(["datadate", "gvkey"])
    .reindex(index)
)
df_prices.head()

ticker  high_raw  low_raw  close_raw     volume  adj_factor  \
date       gvkey                                                                
1999-01-04 001075    PNW    43.375  42.4375     42.875   348400.0         1.0   
           001078    ABT      50.0     48.5      48.75  2226300.0         1.0   
           001161    AMD      29.5   27.625       28.0  2493800.0         2.0   
           001209    APD   39.8125   38.625       39.0   606500.0         1.0   
           001300    HON      44.5  43.3125    43.5625  1417600.0         1.0   

                       trfd  
date       gvkey             
1999-01-04 001075  2.315041  
           001078  1.385097  
           001161  1.000274  
           001209  1.390367  
           001300  1.697016

In [238]:
cols_ffill = ["close_raw", "adj_factor", "trfd", "ticker"]
df_prices[cols_ffill] = df_prices[cols_ffill].ffill(limit=5)
df_prices["volume"] = df_prices["volume"].fillna(0.0)
df_prices["adj_factor"] = df_prices["adj_factor"].fillna(1.0)
df_prices["high_raw"] = df_prices["high_raw"].fillna(df_prices["close_raw"])
df_prices["low_raw"] = df_prices["low_raw"].fillna(df_prices["close_raw"])
df_prices.isna().sum()

ticker        273703
high_raw      273907
low_raw       273907
close_raw     273907
volume             0
adj_factor         0
trfd          295224
dtype: int64

In [239]:
df_prices["is_active"] = df_prices["close_raw"].notna().astype(float)
df_prices.head()

ticker  high_raw  low_raw  close_raw     volume  adj_factor  \
date       gvkey                                                                
1999-01-04 001075    PNW    43.375  42.4375     42.875   348400.0         1.0   
           001078    ABT      50.0     48.5      48.75  2226300.0         1.0   
           001161    AMD      29.5   27.625       28.0  2493800.0         2.0   
           001209    APD   39.8125   38.625       39.0   606500.0         1.0   
           001300    HON      44.5  43.3125    43.5625  1417600.0         1.0   

                       trfd  is_active  
date       gvkey                        
1999-01-04 001075  2.315041        1.0  
           001078  1.385097        1.0  
           001161  1.000274        1.0  
           001209  1.390367        1.0  
           001300  1.697016        1.0

In [273]:
if "volatility" in df_prices.columns:
    mask_bad_price = df_prices["volatility"] >= 1
    print(f"Dropped {mask_bad_price.sum()} rows with Price <= 0")
    df_prices = df_prices[~mask_bad_price]

Dropped 601 rows with Price <= 0


In [274]:
# Adjust for Splits
for col in ["high", "low", "close"]:
    df_prices[f"adj_{col}"] = df_prices[f"{col}_raw"] / df_prices["adj_factor"]

# Intraday Volatility
df_prices["volatility"] = (df_prices["adj_high"] - df_prices["adj_low"]) / (
    df_prices["adj_close"] + 1e-6
)
df_prices["volatility"] = df_prices["volatility"].clip(lower=0.0)

# Log Volume
df_prices["log_volume"] = np.log1p(df_prices["volume"])

# Construct the Total Return Index
# Formula: (Price / SplitFactor) * DividendFactor
# This combines price movement with dividend reinvestment.
df_prices["total_return_index"] = (
    df_prices["close_raw"] / df_prices["adj_factor"]
) * df_prices["trfd"]

# Log Returns
df_prices["log_tridx"] = np.log(df_prices["total_return_index"].replace(0, np.nan))
df_prices["log_ret"] = df_prices.groupby("gvkey")["log_tridx"].diff()
df_prices["log_ret"] = df_prices["log_ret"].fillna(0.0)

upper_limit = np.log(3)  # +200%
lower_limit = np.log(0.1)  # -90%
df_prices["log_ret"] = df_prices["log_ret"].clip(lower=lower_limit, upper=upper_limit)

# Weekly Momentum (5-day Log Return)
df_prices["mom_1w"] = df_prices.groupby("gvkey")["log_ret"].transform(
    lambda x: x.rolling(window=5, min_periods=4).sum()
)

# Monthly Momentum (21-day Log Return)
df_prices["mom_1m"] = df_prices.groupby("gvkey")["log_ret"].transform(
    lambda x: x.rolling(window=21, min_periods=17).sum()
)

# Fill NaNs (The first few days of history will be NaN)
df_prices["mom_1w"] = df_prices["mom_1w"].fillna(0.0)
df_prices["mom_1m"] = df_prices["mom_1m"].fillna(0.0)

In [281]:
cols_price_features = [
    "log_ret",
    "volatility",
    "log_volume",
    "mom_1w",
    "mom_1m",
    "is_active",
]
df_price_features = df_prices[cols_price_features]
df_price_features.head()

log_ret  volatility  log_volume  mom_1w  mom_1m  is_active
date       gvkey                                                             
1999-01-04 001075      0.0    0.021866   12.761109     0.0     0.0        1.0
           001078      0.0    0.030769   14.615852     0.0     0.0        1.0
           001161      0.0    0.066964   14.729319     0.0     0.0        1.0
           001209      0.0    0.030449   13.315462     0.0     0.0        1.0
           001300      0.0     0.02726   14.164477     0.0     0.0        1.0

In [255]:
ratios_query = f"""
SELECT 
    gvkey,
    ticker, public_date,
    ptb,                -- Price to Book (Value)
    pe_exi as pe,       -- P/E Excl. Extraordinary Items (Valuation)
    roe,                -- Return on Equity (Quality)
    de_ratio as dte,    -- Debt to Equity (Leverage)
    divyield            -- Dividend Yield (Income)
FROM wrdsapps.firm_ratio
WHERE gvkey IN {gvkeys_tuple}
AND public_date >= '{FETCH_START_DATE}' -- Fetch earlier to fill 2000 start
ORDER BY ticker, public_date
"""
print("Fetching current S&P 500 constituents financial ratios data from WRDS...")
df_ratios = db_num.raw_sql(ratios_query)

Fetching current S&P 500 constituents financial ratios data from WRDS...


In [317]:
df_ratios["public_date"] = pd.to_datetime(df_ratios["public_date"])
df_ratios = df_ratios.sort_values("public_date")

cols_ratios = ["ptb", "pe", "roe", "dte", "divyield"]
for col in cols_ratios:
    if col in df_ratios.columns:
        df_ratios[col] = df_ratios[col].replace([np.inf, -np.inf], np.nan)
        df_ratios[col] = df_ratios[col].clip(-300, 300)

        if col == "divyield":
            # For dividends, missing usually means 0 (Didn't pay)
            df_ratios[col] = df_ratios[col].fillna(0.0)
        else:
            daily_median = df_ratios.groupby("public_date")[col].transform("median")
            df_ratios[col] = df_ratios[col].fillna(daily_median)
            df_ratios[col] = df_ratios[col].ffill()

In [363]:
df_left = df_price_features.reset_index()
df_pfr = pd.merge_asof(
    df_left,
    df_ratios,
    left_on="date",
    right_on="public_date",
    by="gvkey",
    direction="backward",
    tolerance=pd.Timedelta(days=365),
)
df_pfr = df_pfr.drop(columns=["public_date", "ticker"])
df_pfr = df_pfr.set_index(["date", "gvkey"]).sort_index()

for col in cols_ratios:
    daily_medians = df_pfr.groupby("date")[col].transform("median")
    df_pfr[col] = df_pfr[col].fillna(daily_medians)
    df_pfr[col] = df_pfr[col].fillna(0.0)

In [365]:
df_pfr.head()

log_ret  volatility  log_volume  mom_1w  mom_1m  is_active  \
date       gvkey                                                                
1999-01-04 001075      0.0    0.021866   12.761109     0.0     0.0        1.0   
           001078      0.0    0.030769   14.615852     0.0     0.0        1.0   
           001161      0.0    0.066964   14.729319     0.0     0.0        1.0   
           001209      0.0    0.030449   13.315462     0.0     0.0        1.0   
           001300      0.0     0.02726   14.164477     0.0     0.0        1.0   

                   ptb   pe  roe  dte  divyield  
date       gvkey                                 
1999-01-04 001075  0.0  0.0  0.0  0.0       0.0  
           001078  0.0  0.0  0.0  0.0       0.0  
           001161  0.0  0.0  0.0  0.0       0.0  
           001209  0.0  0.0  0.0  0.0       0.0  
           001300  0.0  0.0  0.0  0.0       0.0

In [346]:
start_macro = pd.to_datetime("1998-01-01")
end_macro = pd.to_datetime("2025-06-01")

# CPIAUCSL: Inflation, UNRATE: Unemployment, PPIACO: Producer Prices
print("Fetching macroeconomic data from FRED...")
df_macro = pdd.DataReader(
    ["CPIAUCSL", "UNRATE", "PPIACO"], "fred", start_macro, end_macro
)

Fetching macroeconomic data from FRED...


In [347]:
df_macro["cpi_yoy"] = df_macro["CPIAUCSL"].pct_change(12).clip(lower=-0.2, upper=0.3)
df_macro["ppi_yoy"] = df_macro["PPIACO"].pct_change(12).clip(lower=-0.2, upper=0.3)
df_macro["unemp_rate"] = df_macro["UNRATE"].div(100).clip(0.0, 0.3)
df_macro["unemp_delta"] = df_macro["UNRATE"].diff()

# LAG CORRECTION: Shift dates +1 Month
df_macro.index = df_macro.index + pd.DateOffset(months=1, days=15)
df_macro = df_macro.reset_index().rename(columns={"DATE": "macro_date"})

In [350]:
df_macro.head()

,macro_date,CPIAUCSL,UNRATE,PPIACO,cpi_yoy,ppi_yoy,unemp_rate,unemp_delta
0,1998-02-16,162.0,4.6,125.4,NaN,NaN,0.046,NaN
1,1998-03-16,162.0,4.6,125.0,NaN,NaN,0.046,0.0
2,1998-04-16,162.0,4.7,124.7,NaN,NaN,0.047,0.1
3,1998-05-16,162.2,4.3,124.9,NaN,NaN,0.043,-0.4
4,1998-06-16,162.6,4.4,125.1,NaN,NaN,0.044,0.1


In [366]:
df_left = df_pfr.reset_index()
df_macro = df_macro.sort_values("macro_date")
df_num = pd.merge_asof(
    df_left.sort_values("date"),
    df_macro[["macro_date", "cpi_yoy", "ppi_yoy", "unemp_rate", "unemp_delta"]],
    left_on="date",
    right_on="macro_date",
    direction="backward",
).drop(columns=["macro_date"])

In [367]:
df_num = df_num.set_index(["date", "gvkey"]).sort_index().loc[START_DATE:]

In [ ]:
df_num.head()

log_ret  volatility  log_volume    mom_1w    mom_1m  \
date       gvkey                                                          
2000-01-03 001075 -0.026945    0.031513    12.41431 -0.045182 -0.114978   
           001078 -0.036814    0.035714   15.378716 -0.060625 -0.062304   
           001161  0.068849    0.058468   15.182011  0.068849  0.058108   
           001209   -0.0226    0.028571   13.219855  0.066212  0.011079   
           001300 -0.017487    0.035281   14.557829  0.041649 -0.069213   

                   is_active       ptb         pe       roe       dte  \
date       gvkey                                                        
2000-01-03 001075        1.0  0.767825   9.987745  0.074962  2.087454   
           001078        1.0  8.369792  23.427419  0.406466  1.139959   
           001161        1.0  2.163137 -16.076389 -0.066714   1.18866   
           001209        1.0  2.084125  16.058612  0.127566  1.737777   
           001300        1.0  7.313876  22.891865   0.31842  1.850449   

                   divyield   cpi_yoy   ppi_yoy  unemp_rate  unemp_delta  
date       gvkey                                                          
2000-01-03 001075  0.045808  0.026204  0.038026       0.041          0.0  
           001078  0.018726  0.026204  0.038026       0.041          0.0  
           001161       0.0  0.026204  0.038026       0.041          0.0  
           001209  0.021453  0.026204  0.038026       0.041          0.0  
           001300  0.011788  0.026204  0.038026       0.041          0.0

In [2]:
df_num.to_parquet("sp500_numerical_data.parquet", index=False)

In [375]:
# Save to Pickle (Native Python Serialization)
df_num.to_pickle("sp500_numerical_data_backup.pkl")

print("Dataframe saved safely to Pickle format.")

Dataframe saved safely to Pickle format.


In [1]:
import pandas as pd

# Reload
df_num = pd.read_pickle("sp500_numerical_data_backup.pkl")